# Задание 1
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [1]:
import pandas as pd

In [8]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [27]:
ratings_grouped = ratings.groupby('userId').count().reset_index()
ratings_grouped.head()
ratings_filtered = ratings_grouped[ratings_grouped.rating > 100]
ratings_filtered.head()

,userId,movieId,rating,timestamp
3,4,204,204,204
7,8,116,116,116
14,15,1700,1700,1700
16,17,363,363,363
18,19,423,423,423


In [56]:
film_fans_user_ids = ratings_filtered['userId'].tolist()
ratings_100 = ratings[ ratings['userId'].isin(film_fans_user_ids) ]
ratings_100.head()

# ratings_100_1 = ratings_100.groupby('userId').agg([min, max])
# ratings_grouped.head()


,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


In [36]:
def max_(df):
    return df.timestamp.max()
def min_(df):
    return df.timestamp.min()
ratings_time_max = ratings_100.groupby('userId').apply(max_)
ratings_time_min = ratings_100.groupby('userId').apply(min_)
ratings_time_max.head()

userId
4      949982274
8     1154474527
15    1469330735
17    1127476640
19     855195373
dtype: int64

In [37]:
ratings_time_min.head()

userId
4      949778714
8     1154389340
15     997937239
17    1127468587
19     855190091
dtype: int64

In [68]:
ratings_time = ratings_time_max - ratings_time_min
ratings_time.head()

userId
4        203560
8         85187
15    471393496
17         8053
19         5282
dtype: int64

In [75]:
ratings_time.mean()/3600/24

463.89476214829165

# Задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

2.1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

2.2. аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:
•rzd - железнодорожные перевозки
•auto - автомобильные перевозки
•air - воздушные перевозки
•client_base - адреса клиентов

In [76]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [77]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [78]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [79]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [87]:
#2.1.
rzd_auto = rzd.merge(auto, on='client_id', how='outer')
rzd_auto_air = rzd_auto.merge(air, on='client_id', how='outer')
rzd_auto_air

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [89]:
#2.2.
revenue_adress = rzd_auto_air.merge(client_base, on='client_id', how='outer')
revenue_adress

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


# Задание 3.
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

In [93]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)
purchases = purchases[['user_id', 'category']]
purchases

,user_id,category
0,11,Спорт
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто


В наличии имеется датафрейм geo с 'user_id', 'location' ([широта,долгота]) (данные в градусах, 6 знаков после запятой).  
Сортировать датафрейм geo по 'user_id' 
1. Создать датафрейм geo_round, равный geo, но с округленными значениями широты и долготы в списке (округление - три знака после запятой для сглаживания погрешности в определении координат с помощью мобильного устройства).
2. Создать из geo_round сводную таблицу со строками 'user_id' и колонками 'location' = geo_round_pivot
3. Датафрейм с визитами и покупками purchases группировать по 'user_id' и посчитать количество вхождений по 'user_id' (count) = visits_grouped
4. Соединить visits_grouped со сводной таблицей из п.2 geo_round_pivot с помощью merge (by='user_id', how='left')

Использовать результат для анализа количества вхождений в различных геоточках.